In [106]:
:ext OverloadedStrings

In [107]:
-- Will print code points instead of actual glyphs
print "你好世界！"

"\20320\22909\19990\30028\65281"

In [108]:
putStrLn "你好世界！"

你好世界！

In [109]:
do
  putStrLn "再见宇宙！"
  putStrLn "开玩笑的"

再见宇宙！
开玩笑的

In [110]:
import Data.Char

In [111]:
ord '我'

25105

In [112]:
putStrLn [chr 25105]

我

In [113]:
import System.Random

In [114]:
-- generate 0 or 1
randomRIO (0, 1)

1

In [115]:
randomRIO (0x4e00, 0x9fff)

27296

In [116]:
import Control.Monad

In [80]:
(: []) . chr <$> randomRIO (0x4e00, 0x9fff) >>= putStrLn

钕

In [11]:
do
  ord <- randomRIO (0x4e00, 0x9fff)
  let hanzi = (: []) $ chr ord
  putStrLn hanzi

労

In [81]:
doN :: Int -> (() -> a) -> [a]
doN = aux []
  -- use helper function to make tail recursive
  where aux acc 0 _fn = acc
        aux acc n fn = aux (fn () : acc) (n - 1) fn

In [75]:
doN 5 (const 8)

[8,8,8,8,8]

In [117]:
randomHanzi :: Int -> IO String
randomHanzi n = sequenceA $ doN n randomChar 
  where randomChar () = chr <$> randomRIO (0x4e00, 0x9fff)

In [118]:
randomHanzi 10 >>= putStrLn

頫獑帖嘵鉕招騢軲埑痔

In [146]:
import IHaskell.Display

do 
  s <- randomHanzi 10
  printDisplay $ html $ "<h1>" ++ s ++ "</h1>"

掯纽飾櫬給郭匏猱咒双